In [30]:
import pyreadr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.regression.linear_model import OLS
from statsmodels.tools import add_constant
from sklearn.linear_model import LinearRegression as lm

embed = lambda x, d : np.hstack([x.shift(i) for i in range(d)])[d - 1:] # R's embed(x, d)

Y = pyreadr.read_r('rawdata.rda')['dados']
nprev=132

In [123]:
def runAR(window, lag, type="fixed"):
    
    # window is given as a DataFrame
    aux = embed(window, lag + 4)
    y = aux[:, 0]
    X = np.delete(aux, range(lag), axis=1)

    if lag == 1:
        out = aux[-1, :X.shape[1]]
    else:
        out = np.delete(aux, range(lag-1), axis=1)
        out = out[-1, :X.shape[1]]

    y = y[:len(y)-lag+1]
    X = X[:len(X)-lag+1]
    cX = add_constant(X, prepend=True)

    if type == "fixed":
        model = OLS(y, cX).fit()
        coef = model.params

    if type == "bic":
        bb = np.inf
        for i in range(X.shape[1]):
            m = OLS(y, add_constant(X[:, i+1], prepend=True)).fit()
            crit = m.bic
            if crit < bb:
                bb = crit
                model = m
                coef = model.params

    # append zeros to coef such that len(coef) == number of col. in cX
    coef = np.pad(coef, (0, (cX.shape[1]-len(coef))), 'constant', constant_values=0)            
    pred = np.concatenate(([1], out), axis=0).dot(coef)
    
    return model, pred, coef


def ar_rolling_window(Y, nprev, indice=0, lag=1, type="fixed", graph=False):
    coef = np.empty((nprev, 5))*np.nan
    pred = np.empty(nprev)*np.nan

    for i in range(nprev):
        window = Y.iloc[i:len(Y)-(nprev-i), [indice]] # The last obs. is not inclued in the last iter.
        model, pred[i], coef[i] = runAR(window, lag)
        #print(f"iteration {i}\n")

    comp = Y.iloc[:, indice].to_frame('actual')
    comp['pred'] = np.concatenate((np.empty(len(Y)-nprev)*np.nan, pred))

    if graph:
        fig, ax = plt.subplots(1,1,figsize=(12, 6))
        comp.plot(ax=ax)
    
    comp = comp.dropna()
    rmse = np.sqrt(np.mean((comp.actual - comp.pred)**2))
    mae = np.mean(np.abs(comp.actual - comp.pred))
    errors = {'rmse':rmse, 'mae':mae}

    return {'pred':comp.pred, 'coef':coef, 'errors':errors}

In [141]:
DF = pd.DataFrame()

for name, index, type in [('cpi', 0, 'fixed'), ('pce', 1, 'fixed'), ('bcpi', 0, 'bic'), ('bpce', 1, 'bic')]:
    df = pd.DataFrame()
    for l in range(12):
        ar = ar_rolling_window(Y, nprev, index, l+1, type)
        df = pd.concat([df, ar['pred'].to_frame(l+1)], axis=1)
    df = pd.concat([df], axis = 1, keys=[name])
    DF = pd.concat([DF, df], axis=1)